# GoodReads Recommendations Data Preparation

## Table of Contents

1. Background
2. Business Understanding
3. Data Understanding
4. Data Preparation
    - 4.a. Book Metadata
    - 4.b. User Review Data
    - 4.c. User Reviews by Book


In [79]:
# Imports
import numpy as np
import pandas as pd
import matplotlib as plt
%matplotlib inline

import re
import ast

from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer

## 1. Background

## 2. Business Understanding

## 3. Data Understanding

## 4. Data Preparation

Note that the datasets are quite large, so they'll take some time to run. I've included comments with average run time for your information.

### 4.a. Book Metadata
I need the metadata of the books in a dataset for two reasons: first, In my collaborative filtering, I will be returning recommendations to users. I'd like to give them information about the books they are recommended. Second, content-based filtering requires finding similar books using their traits/qualities/features. Metadata like the number of pages and the book's description will act as those features.

In [2]:
# This took about 2 mins
cols = ['isbn', 'average_rating', 'similar_books', 'description', 'link', 'authors',
        'num_pages', 'book_id', 'ratings_count', 'title', 'publication_year']
json_reader = pd.read_csv('data/meta_gr.csv', chunksize=10000)

In [3]:
list_df = []
for chunk in json_reader:
    list_df.append(chunk[cols])

In [4]:
metadata = pd.concat(list_df)

In [5]:
metadata.head()

,isbn,average_rating,similar_books,description,link,authors,num_pages,book_id,ratings_count,title,publication_year
0,0312853122,4.00,[],NaN,https://www.goodreads.com/book/show/5333265-w-...,"[{'author_id': '604031', 'role': ''}]",256.0,5333265,3.0,W.C. Fields: A Life on Film,1984.0
1,0743509986,3.23,"['8709549', '17074050', '28937', '158816', '22...","Anita Diamant's international bestseller ""The ...",https://www.goodreads.com/book/show/1333909.Go...,"[{'author_id': '626222', 'role': ''}]",NaN,1333909,10.0,Good Harbor,2001.0
2,NaN,4.03,"['19997', '828466', '1569323', '425389', '1176...",Omnibus book club edition containing the Ladie...,https://www.goodreads.com/book/show/7327624-th...,"[{'author_id': '10333', 'role': ''}]",600.0,7327624,140.0,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",1987.0
3,0743294297,3.49,"['6604176', '6054190', '2285777', '82641', '75...",Addie Downs and Valerie Adler were eight when ...,https://www.goodreads.com/book/show/6066819-be...,"[{'author_id': '9212', 'role': ''}]",368.0,6066819,51184.0,Best Friends Forever,2009.0
4,0850308712,3.40,[],NaN,https://www.goodreads.com/book/show/287140.Run...,"[{'author_id': '149918', 'role': ''}]",NaN,287140,15.0,Runic Astrology: Starcraft and Timekeeping in ...,NaN


In [18]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2360655 entries, 0 to 2360654
Data columns (total 11 columns):
 #   Column            Dtype  
---  ------            -----  
 0   isbn              object 
 1   average_rating    float64
 2   similar_books     object 
 3   description       object 
 4   link              object 
 5   authors           object 
 6   num_pages         float64
 7   book_id           int64  
 8   ratings_count     float64
 9   title             object 
 10  publication_year  float64
dtypes: float64(4), int64(1), object(6)
memory usage: 198.1+ MB


In [ ]:
metadata.isna().sum()

Column-by-column review:
- I will keep 'isbn' and 'link' because it can help users find the exact book when they are recommended it
- I will keep 'average_rating', 'num_pages', 'ratings_count', 'title', and 'publication_year' to act as features in my content-based recommendation model and as descriptors for returned recommendations
- I will keep 'description' to act as a feature in my content-based recommendation model and as a descriiptor for my returned recommendations. It will be NLP processed before entering the content-based model, in the content-based notebook (not here)
- I will keep 'similar_books' as a check for my model's recommendations. The user can see the what the dataset has currently determined are books similar to my model's recommendations
- I will keep 'authors' as a descriptor for my returned recommendations, but I will need to merge in the authors' names using a **separate authors dataset** below
- The 'book_id' column is crucial for connecting this dataset to others

Notably missing here is genre information from the books. I'll merge that in using a **separate genres dataset** below

In [3]:
# Loading in genre data
# 
import pandas as pd
genres = pd.read_json('data/goodreads_book_genres_initial.json', lines=True)
genres.head()

,book_id,genres
0,5333265,"{'history, historical fiction, biography': 1}"
1,1333909,"{'fiction': 219, 'history, historical fiction,..."
2,7327624,"{'fantasy, paranormal': 31, 'fiction': 8, 'mys..."
3,6066819,"{'fiction': 555, 'romance': 23, 'mystery, thri..."
4,287140,{'non-fiction': 3}


In [5]:
genres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2360655 entries, 0 to 2360654
Data columns (total 2 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   book_id  int64 
 1   genres   object
dtypes: int64(1), object(1)
memory usage: 36.0+ MB


In [16]:
genres['genres'][4]

{'non-fiction': 3}

In [10]:
sorted(dict1, key=dict1.get)
first_value = next(iter(my_dict.values()))

{'fiction': 555, 'romance': 23, 'mystery, thriller, crime': 10}

In [54]:

# # Define function to extract top genre
# def extract_top_genre(genre_string):
#     # Remove curly braces and split into individual genre-number pairs
#     genre_pairs = [pair.strip() for pair in genre_string.strip('{}').split(',')]
    
#     # Initialize variables to track top genre and its associated number
#     top_genre = ''
#     max_count = 0
    
#     # Iterate over genre-number pairs
#     for pair in genre_pairs:
#         genre, count = pair.rsplit(':', 1)
#         count = int(count)
#         # Check if count is higher than current max_count
#         if count > max_count:
#             max_count = count
#             top_genre = genre.strip("'")
    
#     return top_genre

# # Apply the function to each row in the 'genres' column and store the result in a new column 'top_genre'
# genres['top_genre'] = genres['genres'].apply(extract_top_genre)

# genres.head()


,book_id,genres,top_genre
0,5333265,"{''history': 1, 'historical fiction': 1, 'biog...",None
1,1333909,"{''fiction': 219': 1, ''history': 1, 'historic...",None
2,7327624,"{''fantasy': 1, 'paranormal': 31': 1, ''fictio...",None
3,6066819,"{''fiction': 555': 1, ''romance': 23': 1, ''my...",None
4,287140,{''non-fiction': 3': 1},None


In [7]:
# Loading in author data
authors = pd.read_json('data/goodreads_book_authors.json', lines=True)
authors.head()

,average_rating,author_id,text_reviews_count,name,ratings_count
0,3.98,604031,7,Ronald J. Fields,49
1,4.08,626222,28716,Anita Diamant,546796
2,3.92,10333,5075,Barbara Hambly,122118
3,3.68,9212,36262,Jennifer Weiner,888522
4,3.82,149918,96,Nigel Pennick,1740


In [8]:
authors = authors.drop(columns=['average_rating', 'text_reviews_count', 'ratings_count'])
authors.head()

,author_id,name
0,604031,Ronald J. Fields
1,626222,Anita Diamant
2,10333,Barbara Hambly
3,9212,Jennifer Weiner
4,149918,Nigel Pennick


In [21]:
authors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829529 entries, 0 to 829528
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   author_id  829529 non-null  int64 
 1   name       829529 non-null  object
dtypes: int64(1), object(1)
memory usage: 12.7+ MB


In [13]:
metadata['authors'][88]

"[{'author_id': '7083555', 'role': ''}]"

In [20]:
# Define a regular expression pattern to match the 'author_id' value
pattern = r"'author_id': '(\d+)'"

# Function to extract the author ID from a string
def extract_author_id(cell):
    # Search for the pattern in the string
    match = re.search(pattern, cell)
    if match:
        # If a match is found, return the extracted author ID as an integer
        return int(match.group(1))
    else:
        # If no match is found, return None or any other appropriate value
        return None

# Apply the function to each cell in the 'author' column
metadata['author_id'] = metadata['authors'].apply(extract_author_id)

# Display the updated DataFrame
metadata.head()


,isbn,average_rating,similar_books,description,link,authors,num_pages,book_id,ratings_count,title,publication_year,author_id
0,0312853122,4.00,[],NaN,https://www.goodreads.com/book/show/5333265-w-...,"[{'author_id': '604031', 'role': ''}]",256.0,5333265,3.0,W.C. Fields: A Life on Film,1984.0,604031.0
1,0743509986,3.23,"['8709549', '17074050', '28937', '158816', '22...","Anita Diamant's international bestseller ""The ...",https://www.goodreads.com/book/show/1333909.Go...,"[{'author_id': '626222', 'role': ''}]",NaN,1333909,10.0,Good Harbor,2001.0,626222.0
2,NaN,4.03,"['19997', '828466', '1569323', '425389', '1176...",Omnibus book club edition containing the Ladie...,https://www.goodreads.com/book/show/7327624-th...,"[{'author_id': '10333', 'role': ''}]",600.0,7327624,140.0,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",1987.0,10333.0
3,0743294297,3.49,"['6604176', '6054190', '2285777', '82641', '75...",Addie Downs and Valerie Adler were eight when ...,https://www.goodreads.com/book/show/6066819-be...,"[{'author_id': '9212', 'role': ''}]",368.0,6066819,51184.0,Best Friends Forever,2009.0,9212.0
4,0850308712,3.40,[],NaN,https://www.goodreads.com/book/show/287140.Run...,"[{'author_id': '149918', 'role': ''}]",NaN,287140,15.0,Runic Astrology: Starcraft and Timekeeping in ...,NaN,149918.0


In [22]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2360655 entries, 0 to 2360654
Data columns (total 12 columns):
 #   Column            Dtype  
---  ------            -----  
 0   isbn              object 
 1   average_rating    float64
 2   similar_books     object 
 3   description       object 
 4   link              object 
 5   authors           object 
 6   num_pages         float64
 7   book_id           int64  
 8   ratings_count     float64
 9   title             object 
 10  publication_year  float64
 11  author_id         float64
dtypes: float64(5), int64(1), object(6)
memory usage: 216.1+ MB


In [26]:
metadata.isna().sum()

isbn                983374
average_rating         524
similar_books            0
description         412249
link                   524
authors                  0
num_pages           764133
book_id                  0
ratings_count          524
title                   12
publication_year    599625
author_id              537
dtype: int64

In [36]:
authors['author_id'] = authors['author_id'].astype(float)

In [37]:
authors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829529 entries, 0 to 829528
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   author_id  829529 non-null  float64
 1   name       829529 non-null  object 
dtypes: float64(1), object(1)
memory usage: 12.7+ MB


In [38]:
# Merge metadata and authors DataFrames based on 'author_id' column
metadata = pd.merge(metadata, authors, how='left', left_on='author_id', right_on='author_id')

metadata.head()


,isbn,average_rating,similar_books,description,link,authors,num_pages,book_id,ratings_count,title,publication_year,author_id,name
0,0312853122,4.00,[],NaN,https://www.goodreads.com/book/show/5333265-w-...,"[{'author_id': '604031', 'role': ''}]",256.0,5333265,3.0,W.C. Fields: A Life on Film,1984.0,604031.0,Ronald J. Fields
1,0743509986,3.23,"['8709549', '17074050', '28937', '158816', '22...","Anita Diamant's international bestseller ""The ...",https://www.goodreads.com/book/show/1333909.Go...,"[{'author_id': '626222', 'role': ''}]",NaN,1333909,10.0,Good Harbor,2001.0,626222.0,Anita Diamant
2,NaN,4.03,"['19997', '828466', '1569323', '425389', '1176...",Omnibus book club edition containing the Ladie...,https://www.goodreads.com/book/show/7327624-th...,"[{'author_id': '10333', 'role': ''}]",600.0,7327624,140.0,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",1987.0,10333.0,Barbara Hambly
3,0743294297,3.49,"['6604176', '6054190', '2285777', '82641', '75...",Addie Downs and Valerie Adler were eight when ...,https://www.goodreads.com/book/show/6066819-be...,"[{'author_id': '9212', 'role': ''}]",368.0,6066819,51184.0,Best Friends Forever,2009.0,9212.0,Jennifer Weiner
4,0850308712,3.40,[],NaN,https://www.goodreads.com/book/show/287140.Run...,"[{'author_id': '149918', 'role': ''}]",NaN,287140,15.0,Runic Astrology: Starcraft and Timekeeping in ...,NaN,149918.0,Nigel Pennick


In [39]:
metadata = metadata.drop(columns=['authors', 'author_id'])
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2360655 entries, 0 to 2360654
Data columns (total 11 columns):
 #   Column            Dtype  
---  ------            -----  
 0   isbn              object 
 1   average_rating    float64
 2   similar_books     object 
 3   description       object 
 4   link              object 
 5   num_pages         float64
 6   book_id           int64  
 7   ratings_count     float64
 8   title             object 
 9   publication_year  float64
 10  name              object 
dtypes: float64(4), int64(1), object(6)
memory usage: 198.1+ MB


In [40]:
# Replacing the 'similar_books' column string of book id's with the corresponding book titles

# Create a dictionary mapping book_id to title
book_id_title_map = dict(zip(metadata['book_id'], metadata['title']))

# Function to replace book_id with title
def replace_with_title(similar_books_str):
    similar_books = eval(similar_books_str)  # Convert string to list of book IDs
    titles = [book_id_title_map.get(int(book_id), '') for book_id in similar_books]  # Get titles for each book ID
    return titles

# Apply the function to each element in the 'similar_books' column
metadata['similar_books'] = metadata['similar_books'].apply(replace_with_title)

metadata.head()


,isbn,average_rating,similar_books,description,link,num_pages,book_id,ratings_count,title,publication_year,name
0,0312853122,4.00,[],NaN,https://www.goodreads.com/book/show/5333265-w-...,256.0,5333265,3.0,W.C. Fields: A Life on Film,1984.0,Ronald J. Fields
1,0743509986,3.23,[Hope Will Find You: My Search for the Wisdom ...,"Anita Diamant's international bestseller ""The ...",https://www.goodreads.com/book/show/1333909.Go...,NaN,1333909,10.0,Good Harbor,2001.0,Anita Diamant
2,NaN,4.03,"[Through a Brazen Mirror, The Hound and the Fa...",Omnibus book club edition containing the Ladie...,https://www.goodreads.com/book/show/7327624-th...,600.0,7327624,140.0,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",1987.0,Barbara Hambly
3,0743294297,3.49,"[The Opposite of Me, Dune Road, A Summer Affai...",Addie Downs and Valerie Adler were eight when ...,https://www.goodreads.com/book/show/6066819-be...,368.0,6066819,51184.0,Best Friends Forever,2009.0,Jennifer Weiner
4,0850308712,3.40,[],NaN,https://www.goodreads.com/book/show/287140.Run...,NaN,287140,15.0,Runic Astrology: Starcraft and Timekeeping in ...,NaN,Nigel Pennick


In [41]:
metadata['similar_books'][2]

['Through a Brazen Mirror',
 'The Hound and the Falcon (The Hound and the Falcon, #1-3)',
 'The Secret Books of Paradys III & IV (Secret Books of Paradys, #3-4)',
 'Child of Light: Mary Shelley',
 'A Blackbird in Silver (Blackbird, #1)',
 'The Youngest Miss Ward',
 'Three to Dorsai!',
 'Lord Darcy  (Lord Darcy, #1-3)',
 'The Many Coloured Land The Golden Torc',
 'Dragonslayer',
 'To the Stars',
 'The Angelic Avengers',
 'Der mechanische Prinz',
 'The Road to Ehvenor (Guardians of the Flame, #6)',
 'Oath of the Renunciates (Darkover #10/13)',
 'Mockingbird',
 'The Far Kingdoms (Anteros, #1)']

In [42]:
# Saving off the metadata dataframe
metadata.to_csv('data/metadata.csv', index=False)

### 4.b. User Review Data

Here we want to end up with: rating, user id, book id, the review text for each user per book

In [55]:
# This took 6 mins to load in
reviews = pd.read_json('data/goodreads_reviews_spoiler.json.gz', compression='gzip', lines=True)


In [56]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1378033 entries, 0 to 1378032
Data columns (total 7 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   user_id           1378033 non-null  object        
 1   timestamp         1378033 non-null  datetime64[ns]
 2   review_sentences  1378033 non-null  object        
 3   rating            1378033 non-null  int64         
 4   has_spoiler       1378033 non-null  bool          
 5   book_id           1378033 non-null  int64         
 6   review_id         1378033 non-null  object        
dtypes: bool(1), datetime64[ns](1), int64(2), object(3)
memory usage: 64.4+ MB


In [57]:
# I only need user_id, review_sentences, rating, and book_id 
reviews_subset = reviews[['user_id', 'rating', 'book_id', 'review_sentences']]
reviews_subset.head()

,user_id,rating,book_id,review_sentences
0,8842281e1d1347389f2ab93d60773d4d,5,18245960,"[[0, This is a special book.], [0, It started ..."
1,8842281e1d1347389f2ab93d60773d4d,3,16981,"[[0, Recommended by Don Katz.], [0, Avail for ..."
2,8842281e1d1347389f2ab93d60773d4d,3,28684704,"[[0, A fun, fast paced science fiction thrille..."
3,8842281e1d1347389f2ab93d60773d4d,0,27161156,"[[0, Recommended reading to understand what is..."
4,8842281e1d1347389f2ab93d60773d4d,4,25884323,"[[0, I really enjoyed this book, and there is ..."


In [58]:
# The review sentences column is a list of each sentence with a number for the index at the beginning
reviews_subset['review_sentences'][2]

[[0, 'A fun, fast paced science fiction thriller.'],
 [0, "I read it in 2 nights and couldn't put it down."],
 [0,
  'The book is about the quantum theory of many worlds which states that all decisions we make throughout our lives basically create branches, and that each possible path through the decision tree can be thought of as a parallel world.'],
 [0,
  'And in this book, someone invents a way to switch between these worlds.'],
 [0, 'This was nicely alluded to/foreshadowed in this quote:'],
 [0, '"I think about all the choices we\'ve made that created this moment.'],
 [0, 'Us sitting here together at this beautiful table.'],
 [0,
  'Then I think of all the possible events that could have stopped this moment from ever happening, and it all feels, I don\'t know..." "What?"'],
 [0, '"So fragile."'],
 [0, 'Now he becomes thoughtful for a moment.'],
 [0,
  'He says finally, "It\'s terrifying when you consider that every thought we have, every choice we could possibly make, branches int

In [59]:
# Function to concatenate sentences and remove numbers and quotes
def concatenate_sentences(sentences):
    concatenated_text = ''
    for sentence_info in sentences:
        # Remove the number at the beginning of each sentence
        sentence = sentence_info[1]
        if sentence_info[0] == 1:
            sentence = sentence[1:]  # Remove the number at the beginning
        # Remove quotes around the sentence
        if sentence.startswith('"') and sentence.endswith('"'):
            sentence = sentence[1:-1]
        # Append the sentence to the concatenated text
        concatenated_text += sentence + ' '
    return concatenated_text.strip()  # Remove trailing space

# Apply the function to concatenate sentences in the 'review_sentences' column
reviews_subset['review_sentences'] = reviews_subset['review_sentences'].apply(concatenate_sentences)
reviews_subset['review_sentences'][2]

/var/folders/45/9490rsy120vgdzpsxzj4d3wr0000gn/T/ipykernel_30889/378747040.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_subset['review_sentences'] = reviews_subset['review_sentences'].apply(concatenate_sentences)


'A fun, fast paced science fiction thriller. I read it in 2 nights and couldn\'t put it down. The book is about the quantum theory of many worlds which states that all decisions we make throughout our lives basically create branches, and that each possible path through the decision tree can be thought of as a parallel world. And in this book, someone invents a way to switch between these worlds. This was nicely alluded to/foreshadowed in this quote: "I think about all the choices we\'ve made that created this moment. Us sitting here together at this beautiful table. Then I think of all the possible events that could have stopped this moment from ever happening, and it all feels, I don\'t know..." "What?" So fragile. Now he becomes thoughtful for a moment. He says finally, "It\'s terrifying when you consider that every thought we have, every choice we could possibly make, branches into a new world." his book can\'t be discussed without spoilers. t is a book about choice and regret. ver 

In [60]:
# Checking that review_sentences has been turned into one string
reviews_subset.head()

,user_id,rating,book_id,review_sentences
0,8842281e1d1347389f2ab93d60773d4d,5,18245960,This is a special book. It started slow for ab...
1,8842281e1d1347389f2ab93d60773d4d,3,16981,Recommended by Don Katz. Avail for free in Dec...
2,8842281e1d1347389f2ab93d60773d4d,3,28684704,"A fun, fast paced science fiction thriller. I ..."
3,8842281e1d1347389f2ab93d60773d4d,0,27161156,Recommended reading to understand what is goin...
4,8842281e1d1347389f2ab93d60773d4d,4,25884323,"I really enjoyed this book, and there is a lot..."


In [61]:
# I want to see how many individual (unique) users there are
user_vc = reviews_subset['user_id'].value_counts()
user_vc

user_id
aca760854b57ce2ec981df32e46dc96c    1815
843a44e2499ba9362b47a089b0b0ce75    1647
8e7e5b546a63cb9add8431ee6914cf59    1214
667b94d4c7e0b014bb6ab3636999e712    1189
c5b70e45e230a166bb00201662495d69    1165
                                    ... 
49e4ac9e5bc4f06039e6120de035738e       1
424fa9e638254075ebe6b4cba7310a91       1
1857263fbefd251fdac5cd6d468aa6e0       1
a0d87052bf1a80b01a9066744fb46e26       1
62e5c281998b211e61dab629ba595ef4       1
Name: count, Length: 18892, dtype: int64

In [62]:
# I want to see how many users I'd be left with if I only kept users who have written at least 5 reviews
users_vc_5 = user_vc[user_vc >=5 ].index
users_vc_5

Index(['aca760854b57ce2ec981df32e46dc96c', '843a44e2499ba9362b47a089b0b0ce75',
       '8e7e5b546a63cb9add8431ee6914cf59', '667b94d4c7e0b014bb6ab3636999e712',
       'c5b70e45e230a166bb00201662495d69', 'aed35dbc626957174ebedf3c555b63d0',
       'd7879573928a367edb1d1accf2372810', 'c0e0fda388f87af0deffad748c9c8b67',
       '422e76592e2717d5d59465d22d74d47c', 'ccf944f8aca6814c1ec21cad667b7123',
       ...
       '54b7ab241624ebbc4158968b48cdd680', 'aaf7d634af94f2c65c04511f80ebdf02',
       '5fcdf0b98fb5d2dea561f5b93b298f8e', '156503c1d86446fd175c965320cb2129',
       'ec4a85bca6494332b52e7257e84b0714', '7ece23a7577ee3377c206ad52a287a47',
       'fdfaa1eeddeb558af311d36eaac263d8', 'c7303bb03f7f4a96c89d44f0504d6b0f',
       '8593c4720b979d9531b023ef8abf0e6d', '58fe4880f1c5746d7311867093747775'],
      dtype='object', name='user_id', length=17268)

The length of the users_vc vs. the users_vc_5 has a difference of only 1,624 users, or 8% of users.

In [64]:
# Keeping only users who have written at least 5 reviews
reviews_subset_2 = reviews_subset[reviews_subset['user_id'].isin(users_vc_5)]

In [65]:
reviews_subset_2['user_id'].value_counts()

user_id
aca760854b57ce2ec981df32e46dc96c    1815
843a44e2499ba9362b47a089b0b0ce75    1647
8e7e5b546a63cb9add8431ee6914cf59    1214
667b94d4c7e0b014bb6ab3636999e712    1189
c5b70e45e230a166bb00201662495d69    1165
                                    ... 
f8b91520e708af9a6c2ea8b2739f28bb       5
3b206409e55a73ca47284a85d84e0cc1       5
fdfaa1eeddeb558af311d36eaac263d8       5
d7c4e0e475bf1421b37890416f4cf1b9       5
297142f776e6cd363a2bc2f7aaa11c81       5
Name: count, Length: 17268, dtype: int64

In [66]:
# We have about 25,500 unique books in the reviews df
reviews_subset_2['book_id'].value_counts()

book_id
11870085    2725
11235712    2276
2767052     2236
7260188     1983
16096824    1716
            ... 
385483         8
2535743        8
16070990       7
118349         7
16148398       1
Name: count, Length: 25475, dtype: int64

In [67]:
reviews_subset_2.info()
# It's fine for user_id to be an object b/c the user id has letters in it

<class 'pandas.core.frame.DataFrame'>
Index: 1373966 entries, 0 to 1378032
Data columns (total 4 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   user_id           1373966 non-null  object
 1   rating            1373966 non-null  int64 
 2   book_id           1373966 non-null  int64 
 3   review_sentences  1373966 non-null  object
dtypes: int64(2), object(2)
memory usage: 52.4+ MB


The review text data will need to be NLP processed before it can be meaningfully used.  Natural Language Processing (NLP) invovles inputs of a certain type: namely, "tokenized" text. Ideally, a string of lower-case, individual, normalized, semantic words. 

Below we initialize a tokenizer, a stopwords list, and a lemmatizer that we use in a custom function. 

The tokenizer will use a regex pattern to turn all words that are at least 3 letters long into a "token."

The stopwords list will be used to remove words like "is" the "the." These are filler words that have no semantic meaning but are still the majority of most speech. They are not useful for prediction and they dramatically increase the input that a model must process. Therefore, our function iterates through the tokenized text and removes them.

Finally our lemmatizer will get to the meaningful "base" or "lemma" of a word. So it will take "change," "changes," "changed," and "changing" and identify them all as the token "change" instead of 4 separate words. This is essentially the "normalization" of text.

In [77]:
# Lower casing
reviews_subset_2['review_sentences']= reviews_subset_2['review_sentences'].apply(lambda x: x.lower())
reviews_subset_2['review_sentences'][2]

/var/folders/45/9490rsy120vgdzpsxzj4d3wr0000gn/T/ipykernel_30889/3263503405.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_subset_2['review_sentences']= reviews_subset_2['review_sentences'].apply(lambda x: x.lower())


'a fun, fast paced science fiction thriller. i read it in 2 nights and couldn\'t put it down. the book is about the quantum theory of many worlds which states that all decisions we make throughout our lives basically create branches, and that each possible path through the decision tree can be thought of as a parallel world. and in this book, someone invents a way to switch between these worlds. this was nicely alluded to/foreshadowed in this quote: "i think about all the choices we\'ve made that created this moment. us sitting here together at this beautiful table. then i think of all the possible events that could have stopped this moment from ever happening, and it all feels, i don\'t know..." "what?" so fragile. now he becomes thoughtful for a moment. he says finally, "it\'s terrifying when you consider that every thought we have, every choice we could possibly make, branches into a new world." his book can\'t be discussed without spoilers. t is a book about choice and regret. ver 

In [80]:
tokenizer = RegexpTokenizer(r"(?u)\w{3,}") # This pattern finds words that are at least 3 letters long
stopwords = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

def preprocessing(text, tokenizer, stopwords, lemmatizer):
    # Tokenize
    tokens = tokenizer.tokenize(text)
    
    # Remove stopwords
    tokens = [token for token in tokens if token not in stopwords]
    
    # Lemmatize
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return tokens

In [81]:
# Apply the preprocessing function to the 'Text' column
reviews_subset_nlp = reviews_subset_2.copy()
reviews_subset_nlp['list_tokens'] = reviews_subset_2['review_sentences'].apply(lambda x: preprocessing(x, tokenizer, stopwords, lemmatizer))
reviews_subset_nlp.head()

,user_id,rating,book_id,review_sentences,list_tokens
0,8842281e1d1347389f2ab93d60773d4d,5,18245960,this is a special book. it started slow for ab...,"[special, book, started, slow, first, third, m..."
1,8842281e1d1347389f2ab93d60773d4d,3,16981,recommended by don katz. avail for free in dec...,"[recommended, katz, avail, free, december, htt..."
2,8842281e1d1347389f2ab93d60773d4d,3,28684704,"a fun, fast paced science fiction thriller. i ...","[fun, fast, paced, science, fiction, thriller,..."
3,8842281e1d1347389f2ab93d60773d4d,0,27161156,recommended reading to understand what is goin...,"[recommended, reading, understand, going, midd..."
4,8842281e1d1347389f2ab93d60773d4d,4,25884323,"i really enjoyed this book, and there is a lot...","[really, enjoyed, book, lot, recommend, drag, ..."


In [82]:
type(reviews_subset_nlp['list_tokens'][0])

list

In [ ]:
reviews_subset_nlp = reviews_subset_nlp.drop(columns='review_sentences')

In [83]:
# Saving the review data
reviews_subset_nlp.to_csv('data/gr_reviews_clean.csv')


### 4.c. User Reviews by Book
Continuation from the user reviews dataset above. We need to join all the reviews for each book into one string. The dataset should have one row per book, with one cell for all that joined review text. This is needed for content-based recommendations, since the review text for each book will act as a feature in the model's search for similar books.

In [88]:
# Turning the tokens into a string
reviews_subset_nlp['string_tokens'] = reviews_subset_nlp['list_tokens'].apply(lambda x: ' '.join(x))

In [89]:
reviews_per_book = reviews_subset_nlp.groupby(['book_id'], as_index=False).agg({'string_tokens': ' '.join})
reviews_per_book.head()

,book_id,string_tokens
0,1,one best book series think get better suspense...
1,2,first read book worst one harry potter series ...
2,3,remember trying time read always gave page ski...
3,5,one definitely good second one much happened r...
4,6,best harry potter book far followed closely bo...


In [91]:
# Locing into the joined reviews for book_id 1
reviews_per_book['string_tokens'][0]

'one best book series think get better suspense build terrific mysterious tale possible double crossing virtuousness delivers best emotional mix good versus evil meta micro scale since ally versus axis second world war first love book since year old got first book christmas hooked still several year book movie theme park later piece always dream magical world hogwarts always home said least favorite book enjoyed learning voldemort past took way much book school year much worth character ron hermione flat lack flavor leave harry alone submerge teen world filled hormone really like suddenly form one book another superficial blind cant care le harry voldemort make sense cried lot end book know happen still hut think knew true beginning knew rowling showing actually allied disguise expecting book full action definitively need happen order give room final book one change everything hoped could fun read book disappoint absolutely love charming aspect book book series sad book series almost g

In [92]:
# Checkingto see that the per-book reviews for book_id 1 match
# all the reviews for book_id 1 in the per-user per book gr_reviews_subset2 dataframe
reviews_subset_nlp[reviews_subset_nlp['book_id']==1]

,user_id,rating,book_id,review_sentences,list_tokens,string_tokens_,string_tokens
901,4672eb229c808b792b8ea95f01f19784,5,1,one of the best books in the series i think.,"[one, best, book, series, think]",one best book series think,one best book series think
1166,5f550cc8d90df1759eda1496621187d8,5,1,it only gets better as the suspense builds in ...,"[get, better, suspense, build, terrific, myste...",get better suspense build terrific mysterious ...,get better suspense build terrific mysterious ...
3027,d8c86e725d683042e9eec675f294d7d4,4,1,"first of all, im in love with these books sinc...","[first, love, book, since, year, old, got, fir...",first love book since year old got first book ...,first love book since year old got first book ...
4408,90c53a2210f33ada709caa5aed0993c2,5,1,this book didn't disappoint me at all. i absol...,"[book, disappoint, absolutely, love, charming,...",book disappoint absolutely love charming aspec...,book disappoint absolutely love charming aspec...
8315,c4f655f2ed79599a9259d7762313f3f3,2,1,urgh.,[urgh],urgh,urgh
...,...,...,...,...,...,...,...
1371766,9de1dd255bb10211df3dfe5152a16f1e,5,1,i really did not see any of what happened in t...,"[really, see, happened, book, coming, beginnin...",really see happened book coming beginning end ...,really see happened book coming beginning end ...
1372411,c979f17afc966aec05933fbcd65c941e,5,1,best book out of the series! it was (one of) m...,"[best, book, series, one, favorite, movie, wel...",best book series one favorite movie well expec...,best book series one favorite movie well expec...
1372854,0f52329fbd7afa66717148ba9dd4ac38,5,1,read (to greg) nov 2005,"[read, greg, nov, 2005]",read greg nov 2005,read greg nov 2005
1377083,a8f8dd7892906b084a27dac976f773f8,5,1,(reread june 2016) still 5 stars!,"[reread, june, 2016, still, star]",reread june 2016 still star,reread june 2016 still star


In [93]:
# Saving off the per-book reviews
reviews_per_book.to_csv('data/gr_reviews_per_book.csv')
